In [1]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
!pip install wikipedia-api
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13467 sha256=6191297741da92b60bbfd57a5f77957fcca491aba9b9650733ba28e0f520a424
  Stored in directory: c:\users\romain\appdata\local\pip\cache\wheels\d3\24\56\58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [3]:
def get_articles(cat_main):
    
    articles_all = []
    
    for p in cat_main.categorymembers.values():

        if p.namespace == wikipediaapi.Namespace.CATEGORY and ('painter' in p.title or 'Painter' in p.title):
            articles_all.extend(get_articles(p))
        
        elif p.namespace == wikipediaapi.Namespace.MAIN:
            articles_all.append(p)

    return articles_all

In [4]:
def get_df_for_cat_value(value_cat_lst, clean_func, valueName):
    
    pages_all = []
    pages_name_all = []
    value_all = []

    for value in tqdm(value_cat_lst):
        
        pages_oneV = get_articles(value)
        pages_name_oneV = [p.title for p in pages_oneV]

        value_clean = clean_func(value)
        value_oneV = [value_clean for p in pages_oneV]

           
        pages_all.extend(pages_oneV)
        pages_name_all.extend(pages_name_oneV)
        value_all.extend(value_oneV)
    
    print('Number of pages before deleting duplicates: ', len(pages_all))
    
    columns = ['page', 'page name', valueName]
    df = pd.DataFrame(columns=columns) 

    df['page'] = pages_all
    df['page name'] = pages_name_all
    df[valueName] = value_all
    
    df = df.groupby('page name').agg({'page':'first', 
                                       valueName: set}).reset_index()
    
    df[valueName] = df[valueName].apply(list)

    
    return df

## Nationalities

In [5]:
def clean_nationality(nat_cat):
    nat = nat_cat.title
    
    nat = nat.split('Category:')[1]
    
    if 'from Georgia' in nat:
        return 'Georgian'
    if 'from the Principality' in nat:
        return 'Liège'
    return nat.split(' painters')[0]

In [6]:
nationalities_cat_lst = []

for nat in  wiki_wiki.page("Category:Painters_by_nationality").categorymembers.values():
    nationalities_cat_lst.append(nat)
    
nationalities_cat_lst = nationalities_cat_lst[8:]

In [7]:
df_nationality = get_df_for_cat_value(value_cat_lst=nationalities_cat_lst, 
                                      clean_func=clean_nationality, 
                                      valueName='nationality')

100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [05:19<00:00,  1.88s/it]


Number of pages before deleting duplicates:  131442


In [8]:
df_nationality

,page name,page,nationality
0,'Abd al-Hayy,"'Abd al-Hayy (id: ??, ns: 0)","[Persian, Iranian]"
1,'the other' Jan van Kessel,"'the other' Jan van Kessel (id: ??, ns: 0)",[Flemish]
2,108 (artist),"108 (artist) (id: ??, ns: 0)",[Italian]
3,3Steps,"3Steps (id: ??, ns: 0)",[German]
4,A. A. Raiba,"A. A. Raiba (id: ??, ns: 0)",[Indian]
...,...,...,...
37045,Živorad Nastasijević,"Živorad Nastasijević (id: ??, ns: 0)",[Serbian]
37046,Ștefan Câlția,"Ștefan Câlția (id: ??, ns: 0)",[Romanian]
37047,Ștefan Dimitrescu,"Ștefan Dimitrescu (id: ??, ns: 0)",[Romanian]
37048,Ștefan Luchian,"Ștefan Luchian (id: ??, ns: 0)",[Romanian]


## Dates and Other Info

In [9]:
def get_dates(page):
    cats = page.categories.keys()

    death_year, birth_year = None, None
    
    for c in cats:

        if 'births' in c:
            match = re.search(r'\d{4}', c) 
            if match: birth_year = match[0]

        if 'deaths' in c:
            match = re.search(r'\d{4}', c) 
            if match: death_year = match[0]

    return [birth_year, death_year, page.fullurl, list(cats)]

In [20]:

    births = []
    deaths = []
    urls = []
    categories_all = []


    for p in tqdm(df_nationality[21719:]['page']):
        b, d, u, cs = get_dates(p)

        births.append(b)
        deaths.append(d)
        urls.append(u)
        categories_all.append(cs)
        
        

100%|██████████████████████████████████████████████████████████████████████████| 15331/15331 [1:41:27<00:00,  2.52it/s]


In [21]:
len(births)

15331

## Other Info

# Save

In [11]:
def clean_title(title):
    if ' (' in title:
        return title.split(' (')[0]
    return title

In [22]:
columns = ['page', 'page name', 'painter name', 'url', 'birth', 'death', 'nationality', 'categories']
df = pd.DataFrame(columns=columns)


df['page'] = df_nationality['page']
df['page name'] = df_nationality['page name']
df['painter name'] = df_nationality['page name'].apply(clean_title)
df['url'] = urls
df['birth'] = births
df['death'] = deaths
df['nationality'] = df_nationality['nationality']
df['categories'] = categories_all


df

,page,page name,painter name,url,birth,death,nationality,categories
21719,"Katharina Rapp (id: 1628554, ns: 0)",Katharina Rapp,Katharina Rapp,https://en.wikipedia.org/wiki/Katharina_Rapp,1948,None,[German],"[Category:1948 births, Category:20th-century G..."
21720,"Katharine Carl (id: 8544717, ns: 0)",Katharine Carl,Katharine Carl,https://en.wikipedia.org/wiki/Katharine_Carl,1865,1938,[American],"[Category:1865 births, Category:1938 deaths, C..."
21721,"Katharine Church (id: 56389959, ns: 0)",Katharine Church,Katharine Church,https://en.wikipedia.org/wiki/Katharine_Church,1910,1999,[British],"[Category:1910 births, Category:1999 deaths, C..."
21722,"Katharine Emma Maltwood (id: 10726642, ns: 0)",Katharine Emma Maltwood,Katharine Emma Maltwood,https://en.wikipedia.org/wiki/Katharine_Emma_M...,1878,1961,[British],"[Category:1878 births, Category:1961 deaths, C..."
21723,"Katharine Lane Weems (id: 11172863, ns: 0)",Katharine Lane Weems,Katharine Lane Weems,https://en.wikipedia.org/wiki/Katharine_Lane_W...,1899,1989,[American],"[Category:1899 births, Category:1989 deaths, C..."
...,...,...,...,...,...,...,...,...
37045,"Živorad Nastasijević (id: 62254857, ns: 0)",Živorad Nastasijević,Živorad Nastasijević,https://en.wikipedia.org/wiki/%C5%BDivorad_Nas...,1893,1966,[Serbian],"[Category:1893 births, Category:1966 deaths, C..."
37046,"Ștefan Câlția (id: 13409891, ns: 0)",Ștefan Câlția,Ștefan Câlția,https://en.wikipedia.org/wiki/%C8%98tefan_C%C3...,1942,None,[Romanian],"[Category:1942 births, Category:All stub artic..."
37047,"Ștefan Dimitrescu (id: 6000470, ns: 0)",Ștefan Dimitrescu,Ștefan Dimitrescu,https://en.wikipedia.org/wiki/%C8%98tefan_Dimi...,1886,1933,[Romanian],"[Category:1886 births, Category:1933 deaths, C..."
37048,"Ștefan Luchian (id: 2475912, ns: 0)",Ștefan Luchian,Ștefan Luchian,https://en.wikipedia.org/wiki/%C8%98tefan_Luchian,1868,1917,[Romanian],"[Category:1868 births, Category:1917 deaths, C..."


In [23]:
df.to_csv('Data_GoogleArts_V1.csv')

## Load Test

In [4]:
df_tmp = pd.read_csv('Data_GoogleArts_V1_21719_37049.csv', index_col=0)
df_tmp.head()

,page,page name,painter name,url,birth,death,nationality,categories
21719,"Katharina Rapp (id: 1628554, ns: 0)",Katharina Rapp,Katharina Rapp,https://en.wikipedia.org/wiki/Katharina_Rapp,1948.0,NaN,['German'],"['Category:1948 births', 'Category:20th-centur..."
21720,"Katharine Carl (id: 8544717, ns: 0)",Katharine Carl,Katharine Carl,https://en.wikipedia.org/wiki/Katharine_Carl,1865.0,1938.0,['American'],"['Category:1865 births', 'Category:1938 deaths..."
21721,"Katharine Church (id: 56389959, ns: 0)",Katharine Church,Katharine Church,https://en.wikipedia.org/wiki/Katharine_Church,1910.0,1999.0,['British'],"['Category:1910 births', 'Category:1999 deaths..."
21722,"Katharine Emma Maltwood (id: 10726642, ns: 0)",Katharine Emma Maltwood,Katharine Emma Maltwood,https://en.wikipedia.org/wiki/Katharine_Emma_M...,1878.0,1961.0,['British'],"['Category:1878 births', 'Category:1961 deaths..."
21723,"Katharine Lane Weems (id: 11172863, ns: 0)",Katharine Lane Weems,Katharine Lane Weems,https://en.wikipedia.org/wiki/Katharine_Lane_W...,1899.0,1989.0,['American'],"['Category:1899 births', 'Category:1989 deaths..."


In [5]:
wiki_wiki = wikipediaapi.Wikipedia('en',extract_format=wikipediaapi.ExtractFormat.WIKI)


In [12]:
df_tmp = df_tmp.reset_index()

In [13]:
re.search(str(int(df_tmp.at[1,'birth'])),wiki_wiki.page(df_tmp.at[1,'page name']).text).start()

37

In [14]:
import json

In [15]:
with open('output_2.json', 'w',encoding='utf-8') as f:
    for j,row in enumerate(df_tmp.iterrows()):
        d = {}
        e = {}
        d['docid'] = "Wikipedia-"+str(j+21719).zfill(5)
        d['doctext'] = wiki_wiki.page(row[1]['page name']).summary
        if np.isnan(row[1]['birth']):
            e['birth'] = []
        else:
            b = re.search(str(int(row[1]['birth'])),d['doctext'])
            if b is not None:
                e['birth'] = [[str(int(row[1]['birth'])),b.start()]]
            else:
                e['birth'] = []
         
        if np.isnan(row[1]['death']):
            e['death'] = []
        else:
            de = re.search(str(int(row[1]['death'])),d['doctext'])
            if de is not None:
                e['death'] = [[str(int(row[1]['death'])),de.start()]]
            else:
                e['death'] = []
            
        l = row[1]['nationality']
        l = l.strip('][').split(", ")
        l_out = []
        for i in l: 
            i = i.strip("'")
            n = re.search(i,d['doctext'])
            if n is not None: 
                l_out.append([i,n.start()])
        e['nationality'] = l_out
        d["extracts"] = e            
        json.dump(d,f,ensure_ascii=False)
        f.write("\n")
        if j%100 == 0:
            print(j)
        

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300


In [18]:
with open('output_cont.json', 'w',encoding='utf-8') as f:
    with open('output.json', 'r',encoding='utf-16') as g1:
        for line in g1:
            f.write(line)
    with open('output_2.json', 'r',encoding='utf-8') as g2:
        for line in g2:
            f.write(line)
    

In [19]:
df_1 = pd.read_csv('Data_GoogleArts_V1_0_21719.csv', index_col=0)
df_2 = pd.read_csv('Data_GoogleArts_V1_21719_37049.csv', index_col=0)
df = pd.concat([df_1,df_2])

In [26]:
l

["'Romanian'"]

In [28]:
df_tmp = df
names = df_tmp['painter name'].to_numpy()
births = df_tmp['birth'].to_numpy()
deaths = df_tmp['death'].to_numpy()
nationalities = df_tmp['nationality'].to_numpy()
g = open('train.json','w',encoding='utf-8')
dev = open('dev.json','w',encoding='utf-8')
test = open('test.json','w',encoding='utf-8')
m = len(names)
a = np.array([0]*int(0.95*m)+[1]*int(0.05*m)+[2]*int(0.05*m))
np.random.shuffle(a)
with open('output_cont.json', 'r',encoding='utf-8') as f:
    c = 0
    for j,i in enumerate(f):
        name  = names[j]
        birth = births[j]
        death = deaths[j]
        nationality = nationalities[j]
        c+=1
        d = json.loads(i)
        for k in d['extracts'].keys():
            if d['extracts'][k]:
                d['extracts'][k] = [d['extracts'][k]]
        n = re.search(name,d['doctext'])
        d["docid"] = "Wikipedia-"+d["docid"].split("-")[-1]
        if n is not None:
            d['extracts']['painter_name'] = [[[name,n.start()]]]
        else:
            d['extracts']['painter_name'] = [[[name,0]]]
        
        #if not d['extracts']['birth'] and not np.isnan(birth):
        #    d['extracts']['birth'] = [[[str(int(birth)),0]]]
    
        #if not d['extracts']['death'] and not np.isnan(death):
        #    d['extracts']['death'] = [[[str(int(death)),0]]]    
        
        #if not d['extracts']['nationality']:
        #    d['extracts']['nationality'] = [[[ind.strip("'"),0] for ind in nationality.strip('][').split(", ")]]
        #else:
        #    for nat in nationality.strip('][').split(", "):
        #        if not nat.strip("'") in [ind for ind,_ in d['extracts']['nationality'][0]]:
        #            d['extracts']['nationality'][0].append([nat.strip("'"),0]) 

        di = {}
        di['1_painter_name'] = d['extracts']['painter_name']
        di['2_birth'] = d['extracts']['birth']
        di['3_death'] = d['extracts']['death']
        di['4_nationality'] = d['extracts']['nationality']
        d['extracts'] = di
        if c>=m:
            break
        if a[c]==0:
            json.dump(d,g,ensure_ascii=False)
            g.write("\n")
        elif a[c]==1:
            json.dump(d,dev,ensure_ascii=False)
            dev.write("\n")
        else:
            json.dump(d,test,ensure_ascii=False)
            test.write("\n")
            
g.close()
dev.close()
test.close()

In [8]:
[i for i in ['Malaysian']]

['Malaysian']

In [15]:
l_nationality = []
with open('train.json','r',encoding='utf-8') as f:
    for line in f:
        d = json.loads(line)
        n = d['extracts']['nationality']
        if n:
            if not n[0][0][0] in l_nationality:
                l_nationality.append(n[0][0][0])

In [37]:
preds = []
with open('test.json','r',encoding='utf-8') as f:
    for line in f:
        d = json.loads(line)
        t = d["doctext"].split(" ")
        name = []
        n_c = True
        nats = []
        birth = 0
        death = 0
        for word in t:
            if word:
                if n_c:
                    if not (word in ["is","was"] or "(" in word):
                        name.append(word)
                    else:
                        n_c = False

                if not n_c:
                    if word[0] == "(":
                        word = word[1:]
                    if word and word[-1] == ")":
                        word = word[:-1]
                    if word in l_nationality and not word in nats:
                        nats.append(word)
                    if word.isnumeric() and len(word)==4:
                        if birth == 0:
                            birth = int(word)
                        elif death == 0:
                            death = int(word)
        pred = {}
        pred["name"] = " ".join(name)
        pred["nationality"] = nats
        pred["birth"] = birth
        pred["death"] = death
        r = 0
        tp = 0
        fp = 0
        if d["extracts"]["painter_name"]:
            r +=1
            if d["extracts"]["painter_name"][0][0][0] in pred["name"] or pred["name"] in d["extracts"]["painter_name"][0][0][0]:
                u =  len(pred["name"])/len(d["extracts"]["painter_name"][0][0][0])
                if u>1:
                    tp+=1/u
                else:
                    tp +=u
        else:
            if pred["name"]:
                fp += 1 
        if d["extracts"]["nationality"]:
            r +=len(d["extracts"]["nationality"][0])
            wolo = []
            for i in range(len(d["extracts"]["nationality"][0])):
                wolo.append(d["extracts"]["nationality"][0][i][0])
                if d["extracts"]["nationality"][0][i][0] in pred["nationality"]:
                    tp+=1
            fp+=max(0,len(pred["nationality"])-len(d["extracts"]["nationality"][0][i]))
        else:
            fp += len(pred["nationality"])
        if d["extracts"]["birth"]:
            r+=1
            if int(d["extracts"]["birth"][0][0][0]) == int(pred["birth"]):
                tp+=1
        elif not pred["birth"] == 0:
            fp +=1
        
        if d["extracts"]["death"]:
            r+=1
            if int(d["extracts"]["death"][0][0][0]) == int(pred["death"]):
                tp+=1
        elif not pred["death"] == 0:
            fp +=1
        fn = r-tp
        pred["prec"] = [tp,fn,fp,tp/(tp+0.5*(fp+fn))]
        preds.append(pred)   
            
            
        

In [39]:
f1_i = 0
f1_cum = 0
tp = 0 
fn = 0 
fp = 0
N = len(preds)
for i in preds:
    f1_i += pred["prec"][-1]
    a , b , c = pred["prec"][:-1]
    tp +=  a
    fn +=  b
    fp +=  c
f1_i/=N
f1_cum = tp/(tp+0.5*(fp+fn))

In [40]:
print(f1_i,f1_cum)

0.6666666666666623 0.6666666666666666
